# Generator
- **要点：**
  - 因为generator expression返回的generator支持iteration protocol，所以generator expression的最常见应用场景式直接当iterable用。相比list comprehension，它的memory efficiency高。

## I. 功能和实现方式

### I.1 两种使用形式
#### 1. generator expression
- 形态：
  1. 和list comprehension很像，只是将<code>[]</code>改成<code>()</code>。
     - 比如：<code>g = (i for i in range(4))</code>
  2. 如果是用在函数里面，常见如：<code>list(), join()</code>,那么generator本身的<code>'()' </code>可以省略。<font color=orange>也就是不用写两层括号。</font>
     - <code>list(i for i in range(4))</code>就是<code>list((i for i in range(4)))</code>
- 功能和用法：
  1. 从使用方法上看，generator expression和list comprehension基本一样。都支持if条件判断，也都支持nested for loops。
  2. 从功能上看的本质区别在于：list comprehension一次生成了所有elements，而generator expression是lazy的，不会一次将所有结果都build到memory中，所以<font color=green>当生成的object很大的时候，generator expression会有明显的**memory efficiency**</font>。

In [19]:
print([x ** 2 for x in range(4)])
print((x ** 2 for x in range(4)))

[0, 1, 4, 9]
<generator object <genexpr> at 0x7775050d2dc0>


#### 2. generator function
- 有两种等价形态：
  1. 用<code>yield</code> statement来返回result的funtion。
     - 比如：
  ```python
  def gen(N):
        for i in range(N):
            yield i
  ```
  2. 用<code>yield from</code> statement来返回result的funtion。
     - 比如：
  ```python
  def gen(N):
        yield from range(N)
  ```

In [20]:
def gen():
    for i in range(5):
        yield i
g = gen()
print(g)

<generator object gen at 0x7775050d03c0>


### I.2 iteration tool + generator如何工作
- generator在计算的时候不会一次完成所有计算，被iteration tool call的时候只会执行到yield statement，此时yield会返回本轮计算的结果，然后保存当前状态，暂停运行。直到再被iteration tool call的时候重新在上次暂停的位置和状态上进行新的一轮计算。
#### I.2.1 generator function的工作过程：iteration tool + generator
1. 在用yield作为返回值定义的function被call之后，会返回一个generator对象。
2. python自动帮generator加上了<code>\_\_iter__</code> 和<code>\_\_next__</code>method，设置<code>\_\_iter__</code>的返回值为对象本身，并根据函数体中的code内容<font color=orange>(从yield后一行到再次执行完yield)</font>来设置<code>\_\_next__</code>的返回值。因此，generator对象支持iteration protocol，使用于所有iteration tools。
3. 每次iteration tool invoke该generator的<code>\_\_next__</code> method后，就会从保存的state信息中调取代码位置和local variable value，并从“上一次执行完的yield statement下一行的位置”重新开始执行函数体中的代码，直到再次遇到<code>yield</code>时，将此次迭代的计算结果返回，然后自动暂停，并存储当前state。
   - 这里存储的state中包括： 
     1. 函数的code location，后面每次generator中的<code>\_\_next__</code> method被call的时候，都还要用到函数定义来执行新的运算。
     2. local scope，函数中的local variable value，每次遍历的时候都要使用。
3. 直到所有的结果都生成完后，generator会自动raise <code>StopIteration</code> exception。

In [18]:
def gensquares(N):
    for i in range(N):
        yield i ** 2

gs = gensquares(5)
print(gs)

for i in gs:
    print(i, end=' ')

<generator object gensquares at 0x7775050d1540>
0 1 4 9 16 

#### I.2.2 generator expression的工作过程：
- 虽然没有yield statement，但执行<code>(func(i) for i in iterable [if cond(i)])</code>的形式的expression的时候，也会返回一个generator object。它在iteration context中的工作方式和generator function返回的generator一样。
- 和generator function的区别在于：
  1. generator expression中函数体部分<code>func(i)</code>只能表达很简单的程序，复杂的程序逻辑还是要用function。
     - <font color=orange>比如：生成fibonacci数列的函数要同时保持两个变量值，这在geberator expression中就不能直接做到。这种情况就应该用generator function，不用expression。</font>
  2. generator expression是一个expression，所以它可以用在一些函数不能用的地方。
     - 比如：下面例子中也可以用list comprehension，但是list会一次生成sequence，当sequence很大的时候，太占memory。
  ```python
  for i in (x ** 2 for x in range(10000)):
        print(2 * i)
  ```

## II. generator的典型应用
- generator function与generator expression的关系：类似于一般function与lambda function的关系
  1. 用function得到的是statement，用expression得到的是expression。expression可以用在statement无法使用的地方
  2. expression适合简单的代码，不适合复杂的代码逻辑

### II.1 generator expression
- 虽然map和filter可以在很多场景下替代generator expression，但后者的代码写起来更简单也更好读。
- 在memory efficiency上，generator expression与map + filter的差异不大，因为map和filter返回的对象也是lazy的
- <font color=green>**因为generator expression返回的generator支持iteration protocol，所以generator expression的最常见应用场景式直接当iterable用。**</font><font color=deeppink>**相比list comprehension，它的memory efficiency高。**</font>
#### 1. map功能
- 两种等价形式：
  - <code>(func for i in iterable)</code>
  - <code>map(func, iterable)</code>

In [27]:
# 简单例子
m = map(abs, range(-2, 3))
print(m)

g = (abs(i) for i in range(-2, 3))
print(g)

list(m) == list(g)

<generator object <genexpr> at 0x777504f77d80>


True

In [26]:
# 需要额外定义函数的例子
m = map(lambda x: x ** 2, range(-2, 3))
g = (i ** 2 for i in range(-2, 3))
list(m) == list(g)

True

#### 2. filter功能
- 两种等价形式：
  - <code>(i for i in iterable if cond(i))</code>
  - <code>filter(cond(i), iterable)</code> <font color=red>注：filter第1个参数是function object</font>

In [38]:
line = 'aa b\n ccc d\n\n'

# 将长度大于1的word取出来存为list
f = filter(lambda x: len(x)>1, line.split()) # split()会自动去掉'\n'
g = (i for i in line.split() if len(i) > 1)

list(mf), list(g)

([], ['aa', 'ccc'])

#### 3. map + filter功能
- 两种等价形式：
  - <code>(func(i) for i in iterable if cond(i))</code>
  - <code>map(func, filter(cond(i), iterable))</code>

In [40]:
# 将长度大于1的word取出来，换成大写字母，再存为list
mf = map(str.upper, filter(lambda x: len(x)>1, line.split()))
g = (i.upper() for i in line.split() if len(i) > 1)

list(mf), list(g)

(['AA', 'CCC'], ['AA', 'CCC'])